In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Customers').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression
dataset=spark.read.csv('Ecommerce_Customers.csv',inferSchema=True,header=True)

In [9]:
dataset
dataset.show(5)
dataset.printSchema()

+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|               Email|             Address|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|       34.49726773|12.65565115|    39.57766802|         4.082620633|         587.951054|
|   hduke@hotmail.com|4547 Archer Commo...|       31.92627203|11.10946073|    37.26895887|         2.664034182|        392.2049334|
|    pallen@yahoo.com|24645 Valerie Uni...|       33.00091476|11.33027806|    37.11059744|         4.104543202|        487.5475049|
|riverarebecca@gma...|1414 David Throug...|       34.30555663|13.71751367|    36.72128268|         3.120178783|         581.852344|
|mstephens@davidso...|14023 Rodriguez P...|       33.33067252|12.79518855|  

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [24]:
featureassembler=VectorAssembler(inputCols=["Avg Session Length","Time on App","Time on Website","Length of Membership"],outputCol='Independent Col')

In [25]:
output=featureassembler.transform(dataset)

In [26]:
output.select('Independent Col').show()

+--------------------+
|     Independent Col|
+--------------------+
|[34.49726773,12.6...|
|[31.92627203,11.1...|
|[33.00091476,11.3...|
|[34.30555663,13.7...|
|[33.33067252,12.7...|
|[33.87103788,12.0...|
|[32.0215955,11.36...|
|[32.73914294,12.3...|
|[33.9877729,13.38...|
|[31.93654862,11.8...|
|[33.99257277,13.3...|
|[33.87936082,11.5...|
|[29.53242897,10.9...|
|[33.19033404,12.9...|
|[32.38797585,13.1...|
|[30.73772037,12.6...|
|[32.1253869,11.73...|
|[32.33889932,12.0...|
|[32.18781205,14.7...|
|[32.61785606,13.9...|
+--------------------+
only showing top 20 rows



In [30]:
finalized_data=output.select("Independent Col","Yearly Amount Spent")

In [31]:
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Col', labelCol='Yearly Amount Spent')
regressor=regressor.fit(train_data)
pred_results=regressor.evaluate(test_data)
pred_results.predictions.show(40)

+--------------------+-------------------+------------------+
|     Independent Col|Yearly Amount Spent|        prediction|
+--------------------+-------------------+------------------+
|[30.4925367,11.56...|        282.4712457| 289.1553223274848|
|[30.57436368,11.3...|        442.0644138| 443.4546366556442|
|[30.83643267,13.1...|        467.5019004| 473.0281666830865|
|[31.06621816,11.7...|        448.9332932|463.14594970437906|
|[31.1695068,13.97...|        427.3565308|417.77869640008635|
|[31.30919264,11.9...|        432.7207178|430.86168417466206|
|[31.36621217,11.1...|        430.5888826|427.81806131579447|
|[31.51473786,12.5...|         489.812488|495.29372527398596|
|[31.57613197,12.5...|         541.226584| 543.9477504753058|
|[31.62536013,13.1...|        376.3369008| 381.1783027980175|
|[31.65480968,13.0...|        475.2634237|468.89002472515244|
|[31.76561882,12.4...|        496.5540816|501.52031391551213|
|[31.8530748,12.14...|        459.2851235| 462.1341876510851|
|[31.864